In [12]:
import baostock as bs
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import pymongo
import json
from Kline import DayKline,WeekKline,MonthKline

rsiOverBuy = 80
rsiOverSell = 20
rsiMiddle = 50

In [50]:
# dataList = []
# dayk = DayKline("sz.000001", "深发展")
# dayK2 = DayKline("sz.000002", "万科A")
# dataList.append(dayk.__dict__)
# dataList.append(dayK2.__dict__)
# dataListJson = json.dumps(dataList, ensure_ascii=False) 
# mydb = connectDB()
# collection = mydb["Kline_Day"]
# collection.insert_many(dataList)
bs.login()
rs = bs.query_stock_basic(code="sz.002030")
stockName = None
if (rs.error_code == '0') & rs.next():
    stockName = rs.get_row_data()[1]
print (stockName)
bs.logout()

login success!
达安基因
logout success!


In [28]:
dataListJson


'[{"code": "sz.000001", "name": "深发展"}, {"code": "sz.000002", "name": "万科A"}]'

In [53]:

##
# 连接数据库，这里用的mongoDB
#
def connectDB():
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient["tinyspark"]
    return mydb
    mycol = mydb["sites"]


##
#  下载日K线数据
#  code 股票代码
#  startDate 起始时间
#  endDate   结束时间
#
def downloadDailyStockKline(bsClient, stockCode, startdate, enddate):
    
    #查股票名字
    #返回示例数据
    #code	code_name	ipoDate	outDate	type	status
    #sh.600000	浦发银行	1999-11-10		1	1

    rs = bsClient.query_stock_basic(code=stockCode)
    stockName = None
    if (rs.error_code == '0') & rs.next():
        stockName = rs.get_row_data()[1]
    if stockName == None:
        raise RuntimeError("无此股票代码：", stockCode)
    
    #### 获取沪深A股历史K线数据 ####
    # 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。
    # 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
    rs = bsClient.query_history_k_data_plus(code,
        "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradeStatus,pctChg,isST",
        start_date=startdate, end_date=enddate,
        frequency="d", adjustflag="3")
    
    #### 打印结果集 ####
    dataList = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
#         data_list.append(rs.get_row_data())
        row = rs.get_row_data()
        klineDay = DayKline(row[1], stockName)
        klineDay.date = row[0]
        klineDay.openPrice = row[2]
        klineDay.highPrice = row[3]
        klineDay.lowPrice = row[4]
        klineDay.closePRice = row[5]
        klineDay.preClosePrice = row[6]
        klineDay.volumn = row[7]
        klineDay.amount = row[8]
        klineDay.adjustflag = row[9]
        klineDay.turn = row[10]
        klineDay.tradeStatus = row[11]
        klineDay.changePercent = row[12]
        klineDay.isST = row[13]
        dataList.append(klineDay.__dict__)

#     dataListJson = json.dumps(dataList, ensure_ascii=False) 
    mydb = connectDB()
    collection = mydb["Kline_Day"]
    if len(dataList) > 0:
        collection.insert_many(dataList)
    else:
        raise RuntimeError("数据为空")
#     result = pd.DataFrame(data_list, columns=rs.fields)

#     # 剔除停盘数据
#     df_status = result[result['tradeStatus'] == '1']

#     df_status['close'] = df_status['close'].astype(float)
    
    return True

##
#  从数据库读取K线数据，转DataFrame
#
def readStockKline(code, period, startDate, endDate):
    

def computeRSI(klineDataFrame):

    rsi_12days = ta.RSI(klineDataFrame['close'],timeperiod=12)
    rsi_6days = ta.RSI(klineDataFrame['close'],timeperiod=6)
    rsi_24days = ta.RSI(klineDataFrame['close'],timeperiod=24)
    klineDataFrame['rsi_6days'] = rsi_6days
    klineDataFrame['rsi_12days'] = rsi_12days
    klineDataFrame['rsi_24days'] = rsi_24days
    ##添加参考线位置
    klineDataFrame['overBuy'] = rsiOverBuy
    klineDataFrame['overSell'] = rsiOverSell
    klineDataFrame['middle'] = rsiMiddle

    # RSI超卖和超买
    rsi_buy_position = klineDataFrame['rsi_6days'] > rsiOverBuy
    rsi_sell_position = klineDataFrame['rsi_6days'] < rsiOverSell
    klineDataFrame.loc[rsi_buy_position[(rsi_buy_position == True) & (rsi_buy_position.shift() == False)].index, '超买'] = '超买'
    klineDataFrame.loc[rsi_sell_position[(rsi_sell_position == True) & (rsi_sell_position.shift() == False)].index, '超卖'] = '超卖'
    return klineDataFrame

    

In [ ]:
def downloadAllKlineDataOfSingleDay(date):
    # 获取指定日期的指数、股票数据
    stock_rs = bs.query_all_stock(date)
    stock_df = stock_rs.get_data()
    data_df = pd.DataFrame()
    for code in stock_df["code"]:
        k_rs = bs.query_history_k_data_plus(code, 
            "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradeStatus,pctChg,isST",
            date, date)
        data_df = data_df.append(k_rs.get_data())
    data_df.to_csv("D:\\demo_assignDayData.csv", encoding="gbk", index=False)


In [54]:
if __name__ == '__main__':
    code = "sz.002030"
    startDate = "2017-01-01"
    endDate = "2019-09-20"
    ## 下载数据
    bs.login()
    result = downloadDailyStockKline(bs, code, startDate, endDate)
    bs.logout()
#     df.to_csv("/Users/matt/Documents/pythonworkspace/dailyKline.csv", encoding='utf_8_sig')
    
#     df = pd.read_csv("/Users/matt/Documents/pythonworkspace/dailyKline.csv")
#     ## 计算开始
#     rsi = computeRSI(df)
#     df2 = rsi[['date','rsi_6days','rsi_12days','rsi_24days','overBuy','overSell','middle']]

#     df2.index = rsi['date']
#     plot = df2.plot(title='RSI', figsize=(40,10))
# #     plt.show()
#     fig = plot.get_figure() 
#     fig.savefig('/Users/matt/Downloads/rsi.png')
#     rsi.to_csv("/Users/matt/Documents/pythonworkspace/rsi.csv",encoding='utf_8_sig')
    

login success!


NameError: name 'df_status' is not defined